# Introduction to PUMS Notebook

The U.S. Census website  contains numerous kinds of demographic  data about the United States.  In this section we focus on data collected in surveys conducted during the decennial  census, which is distinct from the more frequently collected American Community Survey data.  More especifically, we will  confine our attention to the [Census 2000 PUMS data](http://www2.census.gov/census_2000/datasets/PUMS/>). PUMS  stands for **Public Use Microdata Samples**.  It is called *Microdata* because the the records contain information about one person or one household.

The discussion below assumes you have visited the PUMS ages on the Census Bureau site and downloaded at least two files, `revisedpums1_alabama.txt` and the data dictionary available in the Excel workbooks, `5%_PUMS_record_layout.xls`.  For more details see the book draft chapter on data, especially the section on PUMS data.  You also need the Python module `read_in_census_data`, which can also be downloaded from a link in the PUMS data section of the book draft.

## Reading in the data dictionary

First you need to import the `read_in_census_data` module and read in the data dictionary in
the Census web site Excel sheet.

In [1]:
from read_in_census_data import CensusInfo, DataFrameWrapper


one_percent_data_file = 'revisedpums1_alabama_01.txt'
data_dictionary = '5%_PUMS_record_layout.xls'
ci = CensusInfo (data_dictionary)

ModuleNotFoundError: ignored

You can now access information about various variable in the Census data.  You need to supply the variable name (or a legal synonym), the record type (person or household) and the file type (15 or 5%).   Below we look at information for the 'RELATE'
variable, which defines the relationship of a parson to the head of the household.  This is a person record variable (`ci.record_types[1]`) in a 1% file (`ci.file_types[1]`).

In [2]:
ci.print_var_info('RELATE',ci.record_types[1],ci.file_types[1])

NameError: ignored

In the next code snippet, we get a sample line from the alabama file:

In [0]:
line = ci.get_sample_record_from_db (one_percent_data_file)

The next code snippets print info about various variables:

In [0]:
#To get info about a DB variable, 3 things must be specfied
# file type (1 per cent or 5 per cent), record type (person or household)
# and the variable name (column name)
(var0,record_type, file_type) = ('RELATE',ci.record_types[1],ci.file_types[1])
# To get avilable info about the VALUE of a variable (usually an opaque code)
# Get the val of this variable for this line
val = ci.get_db_val(line, var0, record_type)
var_values_info = ci.variable_values_dictionary[record_type][(var0,file_type)]
    
# Print some of the info just retrieved.
print (var0,  record_type, val, var_values_info[val])

#ci.print_var_info ('EDUC', ci.person_record)
ci.print_var_info ('education', ci.person_record)
#ci.print_var_info ('RACE3', ci.person_record)
ci.print_var_info ('race', ci.person_record)
ci.print_var_info ('income', ci.person_record)

# Common to both household and person records, links them
ci.print_var_info('serial_no',ci.household_record)

ci.print_var_info('serial_no',ci.person_record)
ci.print_var_info('num_persons',ci.household_record)

try:
  ci.print_var_info('num_persons',ci.person_record)
except Exception as e:
  print(e.message)

## Use the following code to pick out some variables of interest.

In [0]:
for rt in ci.record_types:
    print(rt)
    print('=' * len(rt))
    print()
    rt_dict = ci.data_dictionary[rt]
    banner = '%-5s %14s  %s' % ('Var', 'Synonym', 'Description')
    print(banner)
    print('=' * len(banner))
    print()
    for var in sorted(rt_dict.keys()):
        syns = ', '.join(ci.backward_syns[var])
        print('%-9s %10s  %s' % (var, syns, rt_dict[var]['DESCRIPTION']))
    print()

## Learn more about the values of variable of interest with the following code.

But remember you have to get the record type right.  The following fails because "INCWS" (Wage/salaray income) is not
a variable for HOUSHOLD records.

In [0]:
ci.print_var_info('INCWS',ci.household_record)

In [0]:
ci.print_var_info('INCWS',ci.person_record)

Values at or above topcode are not directly represented.  Instead the mean for values above topcode (for that particular state) are substituted.  You can find out what these means are for topcoded housing and person records [here](http://webapp1.dlib.indiana.edu/virtual_disk_library/index.cgi/5688252/FID3502/Document/Appendix%20H.txt).  In addition, each variable has a **universe**, a set of entities for which it is supposed to represent information.  For the Wage salary income variable INCWS, that universe is persons 15 years of age or over.  Persons under 15 years of age have a blank in that column.  That represents different information than a 0 (which means a person 15 or older earning no salary income).  What is the right thing to do with such records  when computing average salaries?

Let's do gender next.

In [0]:
ci.print_var_info('gender',ci.person_record)

Many variable are just BOOLEAN variables.  They represent whether or not the person or household falls in some category with a 1 or a 2.  The MENTAL (mental disability) variable shown next is an example.  Variables are sometimes accompanied by **allocation** variables, which give information about how the value of that variable for that particular record was arrived at.  For example the MENTAL variable has an accompanying MENTALA allocation variable, which can have the value 1 or 0, with a 1 indicating that the value of the MENTAL variable was missing in the raw data and was inferred by one of a number of allocation methods outlined [here](https://usa.ipums.org/usa/flags.shtml).  Whether or not allocated data should be excluded is a complicated subject having to do with the methods used for allocation and whether a bias may have been introduced.

In [0]:
ci.print_var_info('MENTAL',ci.person_record)

In [0]:
ci.print_var_info('MENTALA',ci.person_record)

In [0]:
ci.print_var_info('PHYSCL',ci.person_record)

In [0]:
ci.print_var_info('PHYSCLA',ci.person_record)

More verbose data is printed out for a variable that has a complex value set.

In [0]:
ci.print_var_info('education',ci.person_record)

In some cases information about a code value must be gotten by going back to [the Census 2000 PUMS data](http://www2.census.gov/census_2000/datasets/PUMS/)  and downloading some further documentation.  This is true of two very important types of information, occupation and location.  For example, information about the metropolitan area of a household is coded using FIPS MSA/CMSA codes.  The coding explanations are stored in documents available state by state and named according to a standard convention. The name template is "PUMEQ[filetype]-[StateCode].TXT", where filetype is 1 (for 1%) or 5 (for 5%), and "StateCode" is a standard two-letter state code.

So, for example, the file giving all the location code info for the 1% file for Alabmama is named "PUMEQ1-AL.TXT".  So the geographical meaning of code '0040-9360', the MSA/CMSA codes shown below, will be found there.  

In [0]:
ci.print_var_info('MSACMSA1',ci.household_record)

On the other hand, although the record below suggests we look in something called Appendix G for the language codes, they actually appear to be stored in a file called 5%_PUMS_language.xls.  The codes are the same for 1% and 5% data, and so a lot of this auxiliaryu documentation is stored only in the 5% portion of the website.

In [0]:
ci.print_var_info('LANG1',ci.person_record)

## Selecting a subset of the data

In [0]:
df = DataFrameWrapper(ci)
# we select a subset of the columns.
df.fill_frame ([], ci.person_record, one_percent_data_file, ci.one_percent_file, \
                'race','education','income','gender','age','relationship')
df.rows[:5]

Note that the household serial_no field is included automatically, because this is the column
that let us link this to other records (to other persons in the same household, so that
we can aggregate information about households.

In [0]:
df.header

Order of the original rows is preserved so the first two person records are about two people in the
same household.

In [0]:
df.rows[:5]

In [0]:
df.save_frame('alabama_pums_extract.csv',header=True)

In [0]:
import pandas as pd
import numpy as np
## Note default is to use header line to name cols.
## header argument is for selecting cols or renaming cols
## Specify dat types for all the codes that arent really intended to be treated as numbers
## Let pandas try to figure out income,education, and age.
dt_dict ={'serial_no':object,'race':object,'gender':object,'relationship':object}
p_df = pd.read_csv('alabama_pums_extract.csv',
                   dtype=dt_dict)

Just for practice, save this file.  The version we're saving is identical
to the one we just read in.

In [0]:
p_df.to_csv('alabama_pums_extract3.csv',index=False)

In [0]:
p_df

Let's look at a sample record.  Note the use of `iloc`.  Along with `loc`, this is one of
Panda's two most important indexing methods.  It allows both numerical and name based indexing.
Here we just use a number to look at the 279th row:

In [0]:
p_df.iloc[278]

So this is a 31 year old white male head of household (race code 47 =   white alone, gender 1 = male,
relationship 1 = head of household, with an income of $18,600, who completed
one or more years of college but did not receive a degree.

## Appendix: Data types

In [0]:
p_df.dtypes

Notice income,education, and age are integers and floats respectively,  That's good because we're going to want to do aggregation operations like mean and sum on income.  We're going to use the numerical values in education to bin education levels.
Below we single out the class of folks with high school diplomas or above (`EDUCATION` >= 9).

Let's look at how to change data types, which we'll need to do below.  First we cook up a
simple example.

In [0]:
df2 = pd.DataFrame([['1','2'],['34','56']],columns= ['h','t'])
              

df2.dtypes

In [0]:
df2['h'].astype(int)

In [0]:
df2['h'] = df2['h'].astype(int)

In [0]:
df2.dtypes

## Aggregation

In [0]:
p_df

In [0]:
p_df.columns

In [0]:
p_df[p_df['race'] == '47']

We map the values in the `race` column to Booleans using a Boolean test.  We will use resulting Pandas `Series` to create a white/non-white column.

In [0]:
p_df['race'].map(lambda x: x == '47')

In [0]:
p_df['white_only'] = p_df['race'].map(lambda x: x == '47')

In [0]:
p_df

We can now do grouping operations to create a `groupby` series that separates white from non-white households.

In [0]:
grouped0 = p_df['income'].groupby(p_df['white_only'])

In [0]:
grouped0

In [0]:
grouped0.mean()

This number seems rather low.  One reason is we're including those people outside the universe
in the calculation (5 year old children, for example), giving them income 0.
We fix this as follows:

In [0]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['white_only'])

Let's break this apart into pieces.

In [0]:
p_df[p_df['income']>0]

This gives the same 8-column table with records that have no income filtered out, so it's a much shorter table.

In [0]:
p_df[p_df['income']>0]['income']

Now we've got just the income column of that table.  Now we do a `split` step.  For one group 'white_only' = True and for the other,'white_only = False'.

In [0]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['white_only'])

The grouping operation is separated from the step where we apply some operation to the groups to get numbers.  For example, let's take mean.

In [0]:
grouped2.income.mean()

And this time let's sum the incomes in the groups.

In [0]:
grouped2 = p_df.groupby(['white_only'])

In [0]:
totals2 = grouped2.income.sum().fillna(0)

In [0]:
totals2

Since we have only numerical columns we can also do means for all the columns:

In [0]:
totals3 = grouped2.mean().fillna(0)

In [0]:
totals3

A groupby object has keys that correspond to the values in the original column we grouped by.

In [0]:
list(totals.keys())

In [0]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['gender'])

In [0]:
grouped2.mean().fillna(0)

## A more complex example (education)

In [0]:
p_df['white_only'] = p_df['race'].map(lambda x: x == '47')
p_df['high_school'] = p_df['education'].map(lambda x: x >= 9)
p_df2 = p_df.loc[:,['high_school','white_only']]

We make a data frame consisting of just the columns we're interested in using `loc`.

In [0]:
p_df2

We now want counts for the entire data set of each pairing of the high_school diploma
variable with the white_only variable.  This is done with `crosstab`.

In [0]:
xtab = pd.crosstab(p_df2.white_only,p_df2.high_school,margins=True)
xtab

Turning these into percentages turns out to be a little harder than I thought.  Maybe there's
an easier way?  Intuitively, we want to divide each column by the last column, so that all the entries
in the False row are divided by 12689 and all the entries in the True row by 31798 and all the entries
in the All row by 44487.  First we need a version of the last column that is floats rather than integers:

In [0]:
all_f = xtab['All'].map(lambda x: float(x))
all_f

Next we'll just divide each column of `xtab` in turn by `all_f`, resetting each column to
be the resulting column of percentages:

In [0]:
xtab[False] = xtab[False]/all_f
xtab[True] = xtab[True]/all_f
xtab['All'] = xtab['All']/all_f
xtab
#xtab.columns

## Levels of education

The next example is very important.  We make a new column defined via a function that computes something based on the value of  one of the existing columns.  The function is called `education`. It sorts education levels into various bins we will use to define a new column called `degrees`.

In [0]:
def education (x):
    
    if 9 <= x <= 11:
        return 'HS'
    elif x == 12:
        return 'AA'
    elif x == 13:
        return 'BA'
    elif x == 14:
        return 'MA'
    elif x > 14:
        return 'HD'
    else:
        return 'ND'
p_df_inc = p_df[p_df['income']>0]
p_df_inc['degrees'] = p_df_inc['education'].map(education)
p_df_inc2 = p_df_inc.loc[:,['degrees','income']]

In [0]:
p_df_inc2

In [0]:
p_df_inc2.loc[278]

In [0]:
grouped3 = p_df_inc2.groupby(['degrees'])

In [0]:
grouped3.mean()

What degree adds the least value?

## Assignment

### Find the mean income by gender.

### Using section 1.7 ("A more complex example (education)" as a model, construct a table showing the percentages of those receiving a high school education or better for males and females.

## Potential projects

Here are some ideas for potential projects using the PUMS data.

1) Compare income levels for a variety of levels of education state by state. This can be done
   by creating bins for education level as in the example above.  You may wish to explore different
   bins than the ones used in that example.  For this project you must do at least 10 states, and you must
   choose states that represent some variety in population and urbanization.  You will need to turn in the Python
   notebook, some prose and explaining and justifying what you did (2 pages), and you need to produce some graphs
   plotting education level (x axis) and income levels (y axis).  Make sure you can get the data for
   several states on one plot.  Use different colored lines.  Your intellectual goal is  to decide if differences in
   education level explain the differences in income state by state, or whether there are other major factors. An
   alternative visualization is to use maps like the maps we use for the campaign contribution to represent (state by state)
   percentages of residents with a particular level of education reaching a certain income level. 
   
2) Compare income levels for a variety of levels of education for whites and non-whites.  You can do this in
   several states, optionally but you must do at least one.  You will need to turn in the Python
   notebook, some prose and explaining and justifying what you did (2 pages), and you need to produce some graphs
   plotting education level (x axis) and income levels (y axis).  Make sure you can get the data for
   whites and non-whites on one plot.  Use different colored lines.  Your intellectual goal is  to decide if differences in
   education level explain the differences in income for whites and non whites, or whether there are other major factors.
   
   